1.1 Parse Ulta site list of Skincare and Makeup products

In [1]:
import re
import time
import random
import warnings
import pandas as pd
import numpy as np
from typing import List
from lxml import etree as et
from bs4 import BeautifulSoup
from time import sleep

In [2]:
#pip install ediblepickle
#pip install selenium
#pip install webdriver-manager

In [43]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

chrome_options=Options()
chrome_options.add_argument('--no-sandbox')   
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')

warnings.filterwarnings('ignore')
#driver = webdriver.Chrome(ChromeDriverManager().install())
driver = webdriver.Chrome(chrome_options=chrome_options)

In [44]:
#Handle the exception of product without Star Ratings
def get_attr(x,tag,class_name):
    try:
        return x.find(tag,class_name).text
    except Exception:
        return None

In [45]:
#Parse Brand,ProductID,ProductName,Price,Rating from all results page
#Fetch link for each product

def get_product_basic(url):
    i=1
    prod_basic=[]
    while (True):
        page=url+"?page={}".format(i)
        driver.get(page)
        sleep(4)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        prod_listing=soup.find('ul',class_="ProductListingResults__productList ProductListingResults__productList--space--top")
        try:
            prod=prod_listing.find_all('li',class_='ProductListingResults__productCard')    
        except Exception:
            return prod_basic
        
        each_page=[ {"Brand":x.find('span','ProductCard__brand').text,
                     "Name":x.find('span','ProductCard__product').text,
                     "Rating":get_attr(x,'div','ProductCard__rating'),
                     "Price":x.find('div','ProductCard__price').text,  
                     "Link":x.find('a')['href'],
                     "ID":x.find('div','ProductCard')['data-item-id']
                    } for x in prod]
        prod_basic.extend(each_page)
        i=i+1
        if(len(prod)<96): break
    return prod_basic

In [100]:
#Parse list of results page for skincare
skincare_url='https://www.ulta.com/shop/skin-care/all'
all_skincare = get_product_basic(skincare_url)
#print("There are {} skincare products in total.".format(len(all_skincare)))
all_skincare[0]

{'Brand': 'Drunk Elephant',
 'Name': 'D-Bronzi Anti-Pollution Sunshine Drops',
 'Rating': '4.6 out of 5 stars ; 903 reviews(903)',
 'Price': '$38.00 ',
 'Link': 'https://www.ulta.com/p/d-bronzi-anti-pollution-sunshine-drops-pimprod2028056?sku=2588470&dcEvent=true',
 'ID': '2588470'}

In [101]:
df_1=pd.DataFrame.from_dict(all_skincare)
df_1.head()

,Brand,Name,Rating,Price,Link,ID
0,Drunk Elephant,D-Bronzi Anti-Pollution Sunshine Drops,4.6 out of 5 stars ; 903 reviews(903),$38.00,https://www.ulta.com/p/d-bronzi-anti-pollution...,2588470
1,The Ordinary,Glycolic Acid 7% Exfoliating Toning Solution,"4.3 out of 5 stars ; 2130 reviews(2,130)",$13.00,https://www.ulta.com/p/glycolic-acid-7-exfolia...,2551155
2,Estée Lauder,Advanced Night Repair Synchronized Multi-Recov...,"4.5 out of 5 stars ; 18814 reviews(18,814)",$54.00 - $220.00,https://www.ulta.com/p/advanced-night-repair-s...,2568523
3,Dermalogica,Skin Smoothing Cream,"4.5 out of 5 stars ; 1363 reviews(1,363)",Sale Price $23.50$23.50 Original Price $47.00$...,https://www.ulta.com/p/skin-smoothing-cream-pi...,2536765
4,The Ordinary,Niacinamide 10% + Zinc 1% Oil Control Serum,"4.3 out of 5 stars ; 6331 reviews(6,331)",$6.00,https://www.ulta.com/p/niacinamide-10-zinc-1-o...,2551167


In [102]:
df_1.to_csv('Skincare.csv',index=None)

In [46]:
%%time
#Do the same process for skincare
makeup_url="https://www.ulta.com/shop/makeup/all"
all_makeup = get_product_basic(makeup_url)
print("There are {} makeup products in total.".format(len(all_makeup)))
df_2=pd.DataFrame.from_dict(all_makeup)
df_2.to_csv('Makeup.csv',index=None)
df_2.info()

There are 4896 makeup products in total.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4896 entries, 0 to 4895
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Brand   4896 non-null   object
 1   Name    4896 non-null   object
 2   Rating  4670 non-null   object
 3   Price   4896 non-null   object
 4   Link    4896 non-null   object
 5   ID      4896 non-null   object
dtypes: object(6)
memory usage: 229.6+ KB
CPU times: user 23.9 s, sys: 1.9 s, total: 25.8 s
Wall time: 6min 36s


In [47]:
df_2.tail(5)

,Brand,Name,Rating,Price,Link,ID
4891,Smashbox,Cover Shot Eyeshadow Palette: Matte,4.4 out of 5 stars ; 646 reviews(646),$32.00,https://www.ulta.com/p/cover-shot-eyeshadow-pa...,2503595
4892,NARS,Smudge Proof Eyeshadow Base,4.6 out of 5 stars ; 93 reviews(93),$26.00,https://www.ulta.com/p/smudge-proof-eyeshadow-...,2511521
4893,FLOWER Beauty,Miracle Matte Translucent Finishing Powder,3.3 out of 5 stars ; 245 reviews(245),$13.99,https://www.ulta.com/p/miracle-matte-transluce...,2524919
4894,CoverGirl,Flourish by Lash Blast Mascara,"4.3 out of 5 stars ; 1897 reviews(1,897)",$10.99,https://www.ulta.com/p/flourish-by-lash-blast-...,2531617
4895,NYX Professional Makeup,Swear By It Jumbo Eyeshadow Palette,4.5 out of 5 stars ; 96 reviews(96),$35.00,https://www.ulta.com/p/swear-by-it-jumbo-eyesh...,2533282


In [48]:
driver.quit()